In [1]:
import pickle
import pyodbc 
import pandas as pd
# if your output is stored and you want to load it without running the model use the following code:
with open('dbase', 'rb') as f:
    dbase = pickle.load(f)
with open('metadata', 'rb') as f:
    metadat = pickle.load(f)    


In [2]:
import pymssql
from sqlalchemy import create_engine
from sqlalchemy.engine import Engine
from pymssql import Connection
import pandas as pd


# sql server engine func
def get_engine(username:str, password:str, server:str, database:str) -> Engine:
    """ connects to MS SQL Server
    Args:
        username (str)
        password (str)
        server (str)
        database (str)
    Returns:
        Engine
    """
    conn_string = f"mssql+pymssql://{username}:{password}@{server}/{database}"
    engine = create_engine(conn_string)
    return engine

# get connection
def get_cnx(username:str, password:str, server:str, database:str) -> Connection:
    """ enables querying MS SQL Server
    Args:
        username (str): [description]
        password (str): [description]
        server (str): [description]
        database (str): [description]
    Returns:
        pymssql.Connection: [description]
    """
    cnx = pymssql.connect(user=username, password=password,
                          host=server, database=database)
    return cnx

# read table to pandas
def read_sql_table(table_name:str, cnx:pymssql.Connection) -> pd.DataFrame:
    """reads sql table into pandas df
    Args:
        table_name (str): table name in SQL DB
        cnx (pymssql.Connection): connection object used to read query
    Returns:
        pd.DataFrame: [description]
    """
    sql_table = pd.read_sql(sql=f"SELECT * FROM {table_name}",
                            con=cnx)

    return sql_table

# write df tp sql server
def write_df_to_sql(
        expID: str, new_df: pd.DataFrame, table_name: str, cnx: Connection, engine: Engine,
        replace_exist_table=False):
    """[summary]
    #columns_to_consider: List[str]
    Args:
        new_df (pd.DataFrame): name of df to be written to sql db
        table_name (str): name of table to write to 
        cnx (Connection): used to read sql tables
        engine (Engine): used to write to sql db 
        columns_to_consider (List[str]): columns to be examined when checking for duplicates
        init (bool, optional): indicates if tables need 
        to be created for the first time. Defaults to False.
    """
    # add expID
    new_df.insert(0,'ExpID',str(expID))

    # conert all columns to string
    new_df=new_df.applymap(str)

    # convert all columns named with (min) to _min
    new_df.columns=[i.replace('(min)','_min') if '(min)' in i else i for i in new_df.columns]
    
    #write new_df to sql server
    if replace_exist_table:
        if_exists='replace'
    
    else:  
        if_exists='append'

    new_df.to_sql(name=table_name,con=engine, index=False, if_exists=if_exists)
    


In [3]:
engine=get_engine(username='transam1', password='Pass@0@0!', server='usodcvsql0255', database='NAO_PMO_Analytics')
conn=get_cnx(username='transam1', password='Pass@0@0!', server='usodcvsql0255', database='NAO_PMO_Analytics')
Tab=read_sql_table(table_name='zzz_all_forecast', cnx=conn)

In [4]:
import numpy as np
#import the input from SQL Server
tableResult = read_sql_table(table_name='zzz_Forecast_Input', cnx=conn)
df=tableResult['Date'].values
def mod_data(df):
    Map={'01':'Jan','02':'Feb','03':'Mar','04':'Apr','05':'May','06':'Jun'
        ,'07':'Jul','08':'Aug','09':'Sep','10':'Oct','11':'Nov','12':'Dec'}
    arr=[]
    arr1=[]
    for i in range(len(df)):
        yr=str(df[i])[2:4]
        mn=str(df[i])[4:6]
        dt=str(df[i])[6:]
        map_mn=Map[mn]
        arr.append(map_mn+'-'+yr)
        arr1.append(str(df[i])[:4]+'-'+mn+'-'+dt)
    return np.array(arr),np.array(arr1)

tableResult['Date'],tableResult['Date_time']=mod_data(df)
tableResult['Date_time']=tableResult['Date_time'].apply(pd.to_datetime)


In [5]:
tableResult

,Scenario,Product,Sub-Product,Date,Volumes,Date_time
0,Field Ops - Mass Markets - Test 7/7/2021,SOUTHWEST,AZ,Dec-20,34786,2020-12-01
1,Field Ops - Mass Markets - Test 7/7/2021,EAST,NORTHEAST,Jan-19,44454,2019-01-01
2,Field Ops - Mass Markets - Test 7/7/2021,EAST,FL_TX,Oct-19,48903,2019-10-01
3,Field Ops - Mass Markets - Test 7/7/2021,NORTHERN,HEARTLAND,Aug-20,39536,2020-08-01
4,Field Ops - Mass Markets - Test 7/7/2021,NORTHERN,WA_MT,Apr-21,19653,2021-04-01
...,...,...,...,...,...,...
1304,Service Delivery - RBO + SDP + EASE - Te,Wavelengths,Install,Mar-21,429,2021-03-01
1305,Service Delivery - RBO + SDP + EASE - Te,Wavelengths,Install,Apr-21,440,2021-04-01
1306,Service Delivery - RBO + SDP + EASE - Te,Wavelengths,Install,May-21,439,2021-05-01
1307,Service Delivery - RBO + SDP + EASE - Te,Wavelengths,Install,Jun-21,410,2021-06-01


In [6]:
#from test import *
#dbase,metadat,Deleted_dat=par_loop(tableResult)

In [7]:
import distutils.util
from Moldels_build_db import *
#from Neural_net import *
from NN_tunned import *
import pandas as pd
from multiprocessing import cpu_count
from joblib import Parallel,delayed,parallel_backend
from sklearn.preprocessing import MinMaxScaler
from Sarima_par import *
from Holts_par import *
from ets_par import *
Maps={'Jan':'1','Feb':'2','Mar':'3','Apr':'4','May':'5','Jun':'6'
        ,'Jul':'7','Aug':'8','Sep':'9','Oct':'10','Nov':'11','Dec':'12'}
from datetime import datetime
def NA(x):
    if x=='None':
        return None
    else: return x
#all the time series forecasting model
def All_models(data_train,data_test,train_start,train_end):
    MAE={}
    Models={}
    val_array=np.zeros((40,data_test.shape[0]))
    i=0
    dt,ds=data_train,data_test
    scaler=MinMaxScaler(feature_range=(-1, 1))
    scaler=scaler.fit(dt.values.reshape(-1,1))
    data_train_n=scaler.transform(dt.values.reshape(-1,1))
    data_test_n=scaler.transform(ds.values.reshape(-1,1))
    
    starts=str(data_train.index[0])[:10]
    print(data_train)
    print(starts)
    ends= str(pd.date_range(start=starts, periods=len(data_train)+len(data_test)+1,freq='MS')[-1])[:10]
    dtt1=ml_data_test(start=starts,end=ends)
    data_ts=ml_data(dtt1)
    
    #AR model
    mod_ar,err,lg,pred=AR_model(data_train,data_test)
    Models['AR with lags '+str(lg)]=mod_ar
    MAE['AR with lags '+str(lg)]=err
    val_array[i,:]=pred
    i+=1

    #MA model
    mod_ma,err1,lg1,pred=MA_model(data_train,data_test)
    Models['MA with lags '+str(lg1)]=mod_ma
    MAE['MA with lags '+str(lg1)]=err1
    val_array[i,:]=pred
    i+=1
    
    #ARMA model
    mod_arma,err2,lg21,lg22,pred=ARMA_model(data_train,data_test)
    Models['ARMA with lags ('+str(lg21)+','+str(lg22)+')']=mod_arma
    MAE['ARMA with lags ('+str(lg21)+','+str(lg22)+')']=err2
    val_array[i,:]=pred
    i+=1
    
    #ARIMA
    mod_arima,err3,lg31,diff3,lg32,pred=ARIMA_model(data_train,data_test)
    Models['ARIMA with order ('+str(lg31)+','+str(diff3)+','+str(lg32)+')']=mod_arima
    MAE['ARIMA with order ('+str(lg31)+','+str(diff3)+','+str(lg32)+')']=err3
    val_array[i,:]=pred
    i+=1
    
    #Sarima
    if __name__ == '__main__':
        p,d,q,P,D,Q,m=sarima_model(data_train,data_test)
    mod_sarima,err4,lg41,diff4,lg42,a,b,c,f,pred = SARIMA_model(data_train,data_test,p[0],d[0],q[0],P[0],D[0],Q[0],m[0])
    Models['SARMA with order ('+str(lg41)+str(diff4)+str(lg42)+'),('+str(a)+str(b)+str(c)+str(f)+')']=mod_sarima
    MAE['SARMA with order ('+str(lg41)+str(diff4)+str(lg42)+'),('+str(a)+str(b)+str(c)+str(f)+')']=err4
    val_array[i,:]=pred
    i+=1
    
    mod_sarima,err4,lg41,diff4,lg42,a,b,c,f,pred = SARIMA_model(data_train,data_test,p[1],d[1],q[1],P[1],D[1],Q[1],m[1])
    Models['SARMA with order ('+str(lg41)+str(diff4)+str(lg42)+'),('+str(a)+str(b)+str(c)+str(f)+')']=mod_sarima
    MAE['SARMA with order ('+str(lg41)+str(diff4)+str(lg42)+'),('+str(a)+str(b)+str(c)+str(f)+')']=err4
    val_array[i,:]=pred
    i+=1
    
    mod_sarima,err4,lg41,diff4,lg42,a,b,c,f,pred = SARIMA_model(data_train,data_test,p[2],d[2],q[2],P[2],D[2],Q[2],m[2])
    Models['SARMA with order ('+str(lg41)+str(diff4)+str(lg42)+'),('+str(a)+str(b)+str(c)+str(f)+')']=mod_sarima
    MAE['SARMA with order ('+str(lg41)+str(diff4)+str(lg42)+'),('+str(a)+str(b)+str(c)+str(f)+')']=err4
    val_array[i,:]=pred
    i+=1
    
    mod_sarima,err4,lg41,diff4,lg42,a,b,c,f,pred = SARIMA_model(data_train,data_test,p[3],d[3],q[3],P[3],D[3],Q[3],m[3])
    Models['SARMA with order ('+str(lg41)+str(diff4)+str(lg42)+'),('+str(a)+str(b)+str(c)+str(f)+')']=mod_sarima
    MAE['SARMA with order ('+str(lg41)+str(diff4)+str(lg42)+'),('+str(a)+str(b)+str(c)+str(f)+')']=err4
    val_array[i,:]=pred
    i+=1
    
    mod_sarima,err4,lg41,diff4,lg42,a,b,c,f,pred = SARIMA_model(data_train,data_test,p[4],d[4],q[4],P[4],D[4],Q[4],m[4])
    Models['SARMA with order ('+str(lg41)+str(diff4)+str(lg42)+'),('+str(a)+str(b)+str(c)+str(f)+')']=mod_sarima
    MAE['SARMA with order ('+str(lg41)+str(diff4)+str(lg42)+'),('+str(a)+str(b)+str(c)+str(f)+')']=err4
    val_array[i,:]=pred
    i+=1

    #Holt's linear trend
    mod_exp_add,error06,pred=exp_add(data_train,data_test)
    Models["Holts ES with add trend"]=mod_exp_add
    MAE["Holts ES with add trend"]= error06
    val_array[i,:]=pred
    i+=1
    
    #Holt-linear trend with dmaped
    mod_exp_add_ds,error06,pred=mod_exp_add_d(data_train,data_test)
    Models["Holts ES with add damped trend"]=mod_exp_add_ds
    MAE["Holts ES with add damped trend"]= error06
    val_array[i,:]=pred
    i+=1
    
    ##Holt's exponential  trend model
    exp_muls,error07,pred=exp_mul(data_train,data_test)
    Models["Holts ES with mul trend"]=exp_muls
    MAE["Holts ES with mul trend" ]= error07
    val_array[i,:]=pred
    i+=1
    
    ##Holt's exponential  trend model with damped
    exp_mul_dampeds,error07,pred=exp_mul_damped(data_train,data_test)
    Models["Holts ES with mul damped trend"]=exp_mul_dampeds
    MAE["Holts ES with mul damped trend" ]= error07
    val_array[i,:]=pred
    i+=1
    
    ##Holt-Winter’s Seasonal Smoothing additive  model no dmaped
    exp_add_ssns,error06,sp,pred=exp_add_ssn(data_train,data_test)
    Models["Holts ES with add trend & seasonality: " +str(sp)+' periods']=exp_add_ssns
    MAE["Holts ES with add trend & seasonality: " +str(sp)+' periods']= error06
    val_array[i,:]=pred
    i+=1
    
    #Holt-Winter’s Seasonal Smoothing additive model with damped 
    exp_add_ssn_dampeds,error07,sp,pred=exp_add_ssn_damped(data_train,data_test)
    Models["Holts ES with add damped trend and seasonality of " +str(sp)+' periods']=exp_add_ssn_dampeds
    MAE["Holts ES with add damped trend & seasonality: " +str(sp)+' periods']= error07
    val_array[i,:]=pred
    i+=1
    
    #Holt-Winter’s Seasonal Smoothing multiplicative  model no dmaped
    exp_mul_ssns,error08,sp,pred=exp_mul_ssn(data_train,data_test)
    Models["Holts ES with mul trend and seasonality of " +str(sp)+' periods']=exp_mul_ssns
    MAE["Holts ES with mul trend & seasonality: " +str(sp)+' periods']= error08
    val_array[i,:]=pred
    i+=1
    
    #Holt-Winter’s Seasonal Smoothing multiplicative model with damped 
    exp_mul_ssn_dampeds,error09,sp,pred=exp_mul_ssn_damped(data_train,data_test)
    Models["Holts ES with mul damped trend & seasonality: " +str(sp)+' periods']=exp_mul_ssn_dampeds
    MAE["Holts ES with mul damped trend & seasonality: " +str(sp)+' periods']= error09
    val_array[i,:]=pred
    i+=1
    
    #holts tunned model    
    if __name__ == '__main__':
        res=sun_holts(data_train,data_test)
        t,d,s,sp,bc,rb=[],[],[],[],[],[]
        for i in range(len(res)):
            t.append(res[i][0])
            d.append(res[i][1])
            s.append(res[i][2])
            sp.append(res[i][3])
            bc.append(res[i][4])
            rb.append(res[i][5])
        t=[NA(i) for i in t]
        s=[NA(i) for i in s]
        d=[bool(distutils.util.strtobool(i)) for i in d]
        sp=[int(i) for i in sp]
        bc=[bool(distutils.util.strtobool(i)) for i in bc]
        rb=[bool(distutils.util.strtobool(i)) for i in rb]
        
    tunned_holtss,error011,pred=tunned_holts(data_train,data_test,t[0],d[0],s[0],sp[0],bc[0],rb[0])
    Models["Holts tunned model_1: "]=tunned_holtss
    MAE["Holts tunned model_1"]=error011
    val_array[i,:]=pred
    i+=1
    
    tunned_holtss,error011,pred=tunned_holts(data_train,data_test,t[1],d[1],s[1],sp[1],bc[2],rb[1])
    Models["Holts tunned model_2: "]=tunned_holtss
    MAE["Holts tunned model_2"]=error011
    val_array[i,:]=pred
    i+=1
    
    tunned_holtss,error011,pred=tunned_holts(data_train,data_test,t[2],d[2],s[2],sp[2],bc[2],rb[2])
    Models["Holts tunned model_3: "]=tunned_holtss
    MAE["Holts tunned model_3"]=error011
    val_array[i,:]=pred
    i+=1
    
    #Simple Exponential Smoothing (SES)
    Sexp_models,error10,pred=Sexp_model(data_train,data_test)
    Models["Simple Exponential Smoothing"]=Sexp_models
    MAE["Simple Exponential Smoothing"]=error10
    val_array[i,:]=pred
    i+=1
    
    #ETS
    if __name__ == '__main__':
        e,t,s,d,p=[],[],[],[],[]
        res=sun_ets(data_train,data_test)
        for i in range(len(res)):
            e.append(res[i][0])
            t.append(res[i][1])
            s.append(res[i][2])
            d.append(res[i][3])
            p.append(res[i][4])
        e=[NA(i) for i in e]
        t=[NA(i) for i in t]
        s=[NA(i) for i in s]
        d=[bool(distutils.util.strtobool(i)) for i in d]
        p=[int(i) for i in p]
    ets_model,error12,pred=ETS_model(data_train,data_test,e[0],t[0],s[0],p[0],d[0])
    Models["ETS_1"]=ets_model
    MAE["ETS_1"]=error12
    val_array[i,:]=pred
    i+=1
    
    ets_model,error12,pred=ETS_model(data_train,data_test,e[1],t[1],s[1],p[1],d[1])
    Models["ETS_2"]=ets_model
    MAE["ETS_2"]=error12
    val_array[i,:]=pred
    i+=1
    
    ets_model,error12,pred=ETS_model(data_train,data_test,e[2],t[2],s[2],p[2],d[2])
    Models["ETS_3"]=ets_model
    MAE["ETS_3"]=error12
    val_array[i,:]=pred
    i+=1
    
    
    #NN
    model_nn,error,pred=dense_tuned(data_trained=data_train_n,data_tested=data_test_n,sps=1,len_test=len(data_test))
    Models['NN-1']=model_nn
    MAE['NN-1']=error
    val_array[i,:]=scaler.inverse_transform(np.array(pred).reshape(-1,1)).reshape(1,-1)
    
    i+=1
    #LSTM
    model_lstm,error,pred=lstm_tuned(data_trained=data_train_n,data_tested=data_test_n,sps=1,len_test=len(data_test))
    Models['LSTM-1']=model_lstm
    MAE['LSTM-1']=error
    val_array[i,:]=scaler.inverse_transform(np.array(pred).reshape(-1,1)).reshape(1,-1)
    i+=1
    
    #NN
    model_nn,error,pred=dense_tuned(data_trained=data_train_n,data_tested=data_test_n,sps=2,len_test=len(data_test))
    Models['NN-2']=model_nn
    MAE['NN']=error
    val_array[i,:]=scaler.inverse_transform(np.array(pred).reshape(-1,1)).reshape(1,-1)
    i+=1
    
    #CNN
    model_cnn,error,pred=cnn_tuned(data_trained=data_train_n,data_tested=data_test_n,sps=2,len_test=len(data_test))
    Models['CNN-2']=model_cnn
    MAE['CNN-2']=error
    val_array[i,:]=scaler.inverse_transform(np.array(pred).reshape(-1,1)).reshape(1,-1)
    i+=1
    
    #LSTM
    model_lstm,error,pred=lstm_tuned(data_trained=data_train_n,data_tested=data_test_n,sps=2,len_test=len(data_test))
    Models['LSTM-2']=model_lstm
    MAE['LSTM-2']=error
    val_array[i,:]=scaler.inverse_transform(np.array(pred).reshape(-1,1)).reshape(1,-1)
    i+=1
    
    #NN
    model_nn,error,pred=dense_tuned(data_trained=data_train_n,data_tested=data_test_n,sps=3,len_test=len(data_test))
    Models['NN-3']=model_nn
    MAE['NN-3']=error
    val_array[i,:]=scaler.inverse_transform(np.array(pred).reshape(-1,1)).reshape(1,-1)
    i+=1
    
    #CNN
    model_cnn,error,pred=cnn_tuned(data_trained=data_train_n,data_tested=data_test_n,sps=3,len_test=len(data_test))
    Models['CNN-3']=model_cnn
    MAE['CNN-3']=error
    val_array[i,:]=scaler.inverse_transform(np.array(pred).reshape(-1,1)).reshape(1,-1)
    i+=1
    
    #LSTM
    model_lstm,error,pred=lstm_tuned(data_trained=data_train_n,data_tested=data_test_n,sps=3,len_test=len(data_test))
    Models['LSTM-3']=model_lstm
    MAE['LSTM-3']=error
    val_array[i,:]=scaler.inverse_transform(np.array(pred).reshape(-1,1)).reshape(1,-1)
    i+=1
    
    #NN
    model_nn,error,pred=dense_tuned(data_trained=data_train_n,data_tested=data_test_n,sps=4,len_test=len(data_test))
    Models['NN-4']=model_nn
    MAE['NN']=error
    val_array[i,:]=scaler.inverse_transform(np.array(pred).reshape(-1,1)).reshape(1,-1)
    i+=1
    
    #CNN
    model_cnn,error,pred=cnn_tuned(data_trained=data_train_n,data_tested=data_test_n,sps=4,len_test=len(data_test))
    Models['CNN-4']=model_cnn
    MAE['CNN-4']=error
    val_array[i,:]=scaler.inverse_transform(np.array(pred).reshape(-1,1)).reshape(1,-1)
    i+=1
    
    #LSTM
    model_lstm,error,pred=lstm_tuned(data_trained=data_train_n,data_tested=data_test_n,sps=4,len_test=len(data_test))
    Models['LSTM-4']=model_lstm
    MAE['LSTM-4']=error
    val_array[i,:]=scaler.inverse_transform(np.array(pred).reshape(-1,1)).reshape(1,-1)
    i+=1
    
    #LSTM-CNN
    model_lcn,error,pred=lstm_cnn_tuned(data_trained=data_train_n,data_tested=data_test_n,sps=4,len_test=len(data_test))
    Models['CNN-LSTM-4']=model_lcn
    MAE['CNN-LSTM-4']=error
    val_array[i,:]=scaler.inverse_transform(np.array(pred).reshape(-1,1)).reshape(1,-1)
    i+=1
    
    #Prophet
    model_pr,error,pred=prophet_model(data_train,data_test)
    Models['prophet']=model_pr
    MAE['prophet']=error
    val_array[i,:]=pred.reshape(1,-1)
    i+=1
    
    #GPY
    #data_train=data_train.set_index('Date_time')
    #data_test=data_test.set_index('Date_time')
    #starts=str(data_train.index[0])[:10]
    print(data_train)
    print(starts)
    #ends= str(pd.date_range(start=starts, periods=len(data_train)+len(data_test)+1,freq='MS')[-1])[:10]
    dtt1=ml_data_test(start=starts,end=ends)
    data_ts=ml_data(dtt1)
    model_gpy,error,pred=gp_mod(data_train,data_test,data_ts)
    Models['GPy']=model_gpy
    MAE['GPy']=error
    val_array[i,:]=(np.array(pred).reshape(1,-1))
    i+=1
    
    #CART
    dtt1=ml_data_test(start=starts,end=ends)
    data_ts=ml_data(dtt1)
    model_dt,error,pred=Cart_mod(data_train,data_test,data_ts)
    Models['RT']=model_dt
    MAE['RT']=error
    val_array[i,:]=(np.array(pred).reshape(1,-1))
    i+=1
    
    #RF
    dtt1=ml_data_test(start=starts,end=ends)
    data_ts=ml_data(dtt1)
    model_rf,error,pred=Rf_mod(data_train,data_test,data_ts)
    Models['RF']=model_rf
    MAE['RF']=error
    val_array[i,:]=(np.array(pred).reshape(1,-1))
    i+=1
    
    print(MAE)
    return Models,MAE,val_array

#get the predictions from this function
def get_forcasts(models,data_train,data_test,test_start,steps=24):
    names=list(models.keys())
    Res={}
    Flag={}
    data_train=data_train.set_index('Date_time')
    data_test=data_test.set_index('Date_time')
    starts=str(data_test.index[0])[:10]
    scaler=MinMaxScaler(feature_range=(-1, 1))
    scaler=scaler.fit(data_train.values.reshape(-1,1))
    data_train_n=scaler.transform(data_train.values.reshape(-1,1))
    data_test_n=scaler.transform(data_test.values.reshape(-1,1))
    
    ends= str(pd.date_range(start=starts, periods=steps+2,freq='MS')[-1])[:10]
    dtt1=ml_data_test(start=starts,end=ends)
    data_ts=ml_data(dtt1)
    
    i=0
    #AR
    
    Res[names[i]]=models[names[i]].predict(start=test_start, end=test_start+steps)
    i+=1
    
    #MA
    Res[names[i]]=models[names[i]].predict(start=test_start, end=test_start+steps)
    i+=1    
    
    #ARMA
    Res[names[i]]=models[names[i]].predict(start=test_start, end=test_start+steps)
    i+=1
    
    #ARIMA
    Res[names[i]]=models[names[i]].predict(start=test_start, end=test_start+steps)
    i+=1
    
    #SARIMA
    Res[names[i]]=models[names[i]].predict(start=test_start, end=test_start+steps)
    i+=1
    
    #SARIMA
    Res[names[i]]=models[names[i]].predict(start=test_start, end=test_start+steps)
    i+=1
    
    #SARIMA
    Res[names[i]]=models[names[i]].predict(start=test_start, end=test_start+steps)
    #Flag[names[i]]=[sum(1 for number in Res[names[i]] if number < 0)<0 for _ in range(len(Res[names[i]]))]
    i+=1
    #SARIMA
    Res[names[i]]=models[names[i]].predict(start=test_start, end=test_start+steps)
    i+=1
    
    #SARIMA
    Res[names[i]]=models[names[i]].predict(start=test_start, end=test_start+steps)
    i+=1

    #add_Linear
    Res[names[i]]=models[names[i]].forecast(steps=steps+1)
    i+=1
    
    #add_Linear_dampen
    Res[names[i]]=models[names[i]].forecast(steps=steps+1)
    #Flag[names[i]]=[sum(1 for number in Res[names[i]] if number < 0)<0 for _ in range(len(Res[names[i]]))]
    i+=1
    
    #mul_Linear
    Res[names[i]]=models[names[i]].forecast(steps=steps+1)
    i+=1
    
    #mul_Linear_dampen
    Res[names[i]]=models[names[i]].forecast(steps=steps+1)
    i+=1
    
    #add_Linear_ssn
    Res[names[i]]=models[names[i]].forecast(steps=steps+1)
    i+=1
    
    #add_linear_dampen_ssn
    Res[names[i]]=models[names[i]].forecast(steps=steps+1)
    i+=1
    
    #mul_linear_ssn
    Res[names[i]]=models[names[i]].forecast(steps=steps+1)
    i+=1
    
    #mul_linear_dampen_ssn
    Res[names[i]]=models[names[i]].forecast(steps=steps+1)
    i+=1
    
    #holt's_tunned_model
    Res[names[i]]=models[names[i]].predict(start=test_start, end=test_start+steps)
    i+=1
    
    #holt's_tunned_model
    Res[names[i]]=models[names[i]].predict(start=test_start, end=test_start+steps)
    i+=1
    
    #holt's_tunned_model
    Res[names[i]]=models[names[i]].predict(start=test_start, end=test_start+steps)
    i+=1
    
    #SES
    Res[names[i]]=models[names[i]].predict(start=test_start, end=test_start+steps)
    i+=1
    
    #ETS
    Res[names[i]]=models[names[i]].forecast(steps=steps+1)
    i+=1
    
    #ETS
    Res[names[i]]=models[names[i]].forecast(steps=steps+1)
    i+=1
    
    #ETS
    Res[names[i]]=models[names[i]].forecast(steps=steps+1)
    i+=1
    
    #Dense
    Res[names[i]]= scaler.inverse_transform(np.array(Forcast(models[names[i]],data_test_n,steps=steps+1,method="NN",sps=1)).reshape(-1,1))
    i+=1
    
    #LSTM
    Res[names[i]]=scaler.inverse_transform(np.array(Forcast(models[names[i]],data_test_n,steps=steps+1,method='LSTM',sps=1)).reshape(-1,1))
    #Flag[names[i]]=[sum(1 for number in Res[names[i]] if number < 0)<0 for _ in range(len(Res[names[i]]))]
    i+=1
    
    #Dense
    Res[names[i]]=scaler.inverse_transform(np.array(Forcast(models[names[i]],data_test_n,steps=steps+1,method="NN",sps=2)).reshape(-1,1))
    i+=1
    
    #CNN
    Res[names[i]]=scaler.inverse_transform(np.array(Forcast(models[names[i]],data_test_n,steps=steps+1,method="CNN",sps=2)).reshape(-1,1))
    i+=1
    
    #LSTM
    Res[names[i]]=scaler.inverse_transform(np.array(Forcast(models[names[i]],data_test_n,steps=steps+1,method='LSTM',sps=2)).reshape(-1,1))
    i+=1
    
    #Dense
    Res[names[i]]=scaler.inverse_transform(np.array(Forcast(models[names[i]],data_test_n,steps=steps+1,method="NN",sps=3)).reshape(-1,1))
    i+=1
    
    #CNN
    Res[names[i]]=scaler.inverse_transform(np.array(Forcast(models[names[i]],data_test_n,steps=steps+1,method="CNN",sps=3)).reshape(-1,1))
    i+=1
    
    #LSTM
    Res[names[i]]=scaler.inverse_transform(np.array(Forcast(models[names[i]],data_test_n,steps=steps+1,method='LSTM',sps=3)).reshape(-1,1))
    i+=1
    
    #Dense
    Res[names[i]]=scaler.inverse_transform(np.array(Forcast(models[names[i]],data_test_n,steps=steps+1,method="NN",sps=4)).reshape(-1,1))
    i+=1
    
    #CNN
    Res[names[i]]=scaler.inverse_transform(np.array(Forcast(models[names[i]],data_test_n,steps=steps+1,method="CNN",sps=4)).reshape(-1,1))
    i+=1
    
    #LSTM
    Res[names[i]]=scaler.inverse_transform(np.array(Forcast(models[names[i]],data_test_n,steps=steps+1,method='LSTM',sps=4)).reshape(-1,1))
    i+=1
    
    #CNN-LSTM
    Res[names[i]]=scaler.inverse_transform(np.array(Forcast(models[names[i]],data_test_n,steps=steps+1,method="LSTM_CNN",sps=4)).reshape(-1,1))
    i+=1
    
    #Prophet
    dt=models[names[i]].make_future_dataframe(periods=steps+1, freq = "M",include_history=False)
    Res[names[i]]=models[names[i]].predict(dt)['yhat']
    i+=1
    
    #Gpy
    Res[names[i]]=models[names[i]].predict(data_ts)
    i+=1
    
    #CART
    Res[names[i]]=models[names[i]].predict(data_ts)
    i+=1
    
    #RF
    Res[names[i]]=models[names[i]].predict(data_ts)
    i+=1
    
    return Res,[starts,ends]#,Flag

#for senity check: 
def Assert(x,y):
    for i in range(len(x)):
        if x[i]==y[i]:
            continue
        else:
            False
    return True

#
def get_excel_pred(prudct_A_x_models,prudct_A_x_mad,val_array,data_train,data_test,test_start,steps=24):
    Res,send=get_forcasts(prudct_A_x_models,data_train,data_test,test_start,steps=steps-1)
    names=list(prudct_A_x_models.keys())
    n_methods=len(names)
    array=np.zeros((n_methods,Res[names[0]].shape[0]))
    
    for i in range(n_methods):
        try:
            array[i,:]=Res[names[i]].reshape(1,-1)
            print(Assert(val_array[i,:],array[i,:len(data_test)]))
            print('ok: '+str(i))
        except Exception as e:
            array[i,:]=Res[names[i]].values.reshape(1,-1)
            print(Assert(val_array[i,:],array[i,:len(data_test)]))
            print('ok: '+str(i))
    
    return array,send


def par_loop(tableResult,train_start='2019-01-01',train_end='2020-12-01'):
    scenario=tableResult['Scenario'].unique()
    Database=[]
    metadata=[]
    Deleted_dat=[]
    for isc in range(len(scenario)):
        product=tableResult.loc[tableResult['Scenario']==scenario[isc]]['Product'].unique()
        for jpr in range(len(product)):
            subproduct=tableResult.loc[(tableResult['Scenario']==scenario[isc]) \
            & (tableResult['Product']==product[jpr])]['Sub-Product'].unique()
            for ksub in range(len(subproduct)):
                print([scenario[isc],product[jpr],subproduct[ksub]])
                Data=tableResult.loc[(tableResult['Scenario']==scenario[isc]) \
                                    & (tableResult['Product']==product[jpr])\
                                        & (tableResult['Sub-Product']==subproduct[ksub])][['Date_time','Date','Volumes']]
                Data=Data.sort_values(by='Date_time')
                Data.reset_index(drop=True)
                Data=Data[['Date_time','Volumes']]
                Data=Data.set_index('Date_time')
                if len(Data)<12:
                    Deleted_dat.append([scenario[isc],product[jpr],subproduct[ksub]])
                    continue
                trains_start=pd.Timestamp(train_start)
                trains_end=pd.Timestamp(train_end)
                if Data.index[0]>trains_start:
                    days=days_between(train_start, str(Data.index[0])[:10])
                    if days<0:#means Data have starting extra values
                        st=abs(days)//29
                        Data=Data[st:]
                    elif days>0: #data misssing some values
                        st=abs(days)//29
                        date_index=pd.date_range(start=train_start, end=Data.index[0],freq='MS')[:-1]
                        val=list(Data.mean())*st
                        df1=pd.DataFrame({'Date_time':date_index,'Volumes':val})
                        df1=df1.set_index('Date_time')
                        Data=df1.append(Data[st:])
                    else:pass
                if Data.index[-1]<trains_end: #have to define here regrading when to fininsh the training
                    days=days_between(train_end, str(Data.index[-1])[:10])
                    if days>0:#means Data have starting extra values
                        st=abs(days)//29
                        Data=Data[st:]
                    elif days<0: #data misssing some values
                        st=abs(days)//29
                        date_index=pd.date_range(start=Data.index[-1], end=train_end,freq='MS')[1:]
                        #date_index=[str(date_index[i])[:10] for i in range(len(date_index))]
                        val=list(Data.mean())*st
                        df1=pd.DataFrame({'Date_time':date_index,'Volumes':val})
                        df1=df1.set_index('Date_time')
                        Data1=Data1.append(df1)
                    else:pass
                data_train,data_test=Data[train_start:train_end],Data[train_end:][1:]
                Ln=data_train.shape[0]
                print('train data shape  {} and test shape is {}'.format(data_train.shape[0],data_test.shape[0]))
                if __name__ == '__main__':
                    prudct_A_x_models,prudct_A_x_mad,val_array=All_models(data_train,data_test,train_start=train_start,train_end=train_end)
                    arr,send=get_excel_pred(prudct_A_x_models,prudct_A_x_mad,val_array,data_train,data_test,test_start=len(data_train),steps=24)                     
                    Database.append(arr)
                    metadata.append([scenario[isc],product[jpr],subproduct[ksub],send,data_test.shape[0]])

    return Database,metadata,Deleted_dat

#have to denote the training start date and training end date from here:
dbase,metadat,Deleted_dat=par_loop(tableResult,train_start='2019-01-01',train_end='2020-12-01')


['Field Ops - Mass Markets - Test 7/7/2021', 'SOUTHWEST', 'AZ']
train data shape  24 and test shape is 5
            Volumes
Date_time          
2019-01-01    45565
2019-02-01    39094
2019-03-01    41412
2019-04-01    39175
2019-05-01    39036
2019-06-01    40985
2019-07-01    46025
2019-08-01    49240
2019-09-01    43530
2019-10-01    43962
2019-11-01    40116
2019-12-01    39700
2020-01-01    40182
2020-02-01    34040
2020-03-01    38672
2020-04-01    43733
2020-05-01    40684
2020-06-01    41621
2020-07-01    44784
2020-08-01    48016
2020-09-01    42185
2020-10-01    39760
2020-11-01    34250
2020-12-01    34786
2019-01-01
The AR model have the lag: 1 & MAE: 8896.603127891478
MA model have the lag: 3 & MAE: 9958.629086042472
ARMA with lags (1 , 0) & MAE: 9455.847634251735
ARIMA with order (022) MAE: 793.3706028479224
DONE!
(659.3229132140754, (1, 2, 1), (0, 1, 1, 10))
         Current function value: 5.029031
         Iterations: 50
         Function evaluations: 126
         Grad

[("['mul', 'add', 'mul', False, 8]", 1567.617080021098), ("['add', 'add', 'mul', False, 8]", 1612.3673442210252), ("['mul', None, 'add', False, 8]", 1814.0668339997478), ("['mul', None, 'add', False, 6]", 2496.090872841562), ("['add', None, 'add', False, 6]", 2975.3401685358126), ("['mul', 'add', 'add', True, 8]", 3153.8835211325504), ("['add', 'add', 'mul', False, 4]", 3459.927449895096), ("['mul', 'add', 'mul', False, 4]", 3499.4529725168422), ("['add', 'add', 'mul', False, 12]", 3873.4482509093855), ("['add', 'add', 'mul', True, 12]", 3910.4735671050244), ("['add', None, 'mul', False, 12]", 3963.570239582231), ("['mul', 'add', 'mul', False, 12]", 4001.852949157509), ("['add', 'add', 'add', False, 8]", 4029.8246006147647), ("['mul', 'add', 'mul', True, 12]", 4038.0157711591464), ("['mul', None, 'mul', False, 12]", 4090.879973816438), ("['mul', None, 'add', False, 7]", 4140.283345451894), ("['mul', 'add', 'mul', True, 2]", 4317.055361157308), ("['mul', 'add', None, True, 2]", 4371.120

ETS error:1814.0668339997478
#######################################################################
Neural Net with MAD is: 1.2491300605033575
LSTM with MAD is: 1.247462936470383
#######################################################################
Neural Net with MAD is: 1.338683284053677
CNN with MAD is: 1.3819949172233281
LSTM with MAD is: 1.360525531301373
#######################################################################
Neural Net with MAD is: 1.5192223764752089
CNN with MAD is: 1.607871726879948
LSTM with MAD is: 1.376045387217873
#######################################################################
Neural Net with MAD is: 1.5125518020038542
CNN with MAD is: 1.476003864945942
LSTM with MAD is: 1.4311339352075991
LSTM+CNN Net with MAE is: 1.394549373516911


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 18.


The prophet model with MAD is: 8836.122797777156
    Date_time  Volumes
0  2019-01-01    45565
1  2019-02-01    39094
2  2019-03-01    41412
3  2019-04-01    39175
4  2019-05-01    39036
5  2019-06-01    40985
6  2019-07-01    46025
7  2019-08-01    49240
8  2019-09-01    43530
9  2019-10-01    43962
10 2019-11-01    40116
11 2019-12-01    39700
12 2020-01-01    40182
13 2020-02-01    34040
14 2020-03-01    38672
15 2020-04-01    43733
16 2020-05-01    40684
17 2020-06-01    41621
18 2020-07-01    44784
19 2020-08-01    48016
20 2020-09-01    42185
21 2020-10-01    39760
22 2020-11-01    34250
23 2020-12-01    34786
2019-01-01
GPY: 10814.441666666664
[[0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 

Holts tunned model: 962.9713179599363
Holts tunned model: 966.2869764149007
Simple EXP MAD: 3376.9558933146996
[("['add', None, 'mul', False, 8]", 1036.3697729046683), ("['add', 'add', 'add', False, 9]", 1120.0682533287313), ("['add', 'add', 'mul', False, 4]", 1146.0419464721024), ("['mul', 'add', 'add', False, 5]", 1219.5245368661017), ("['add', 'add', 'add', False, 5]", 1227.238023234059), ("['mul', 'add', 'mul', False, 4]", 1266.054366294945), ("['add', None, 'add', False, 6]", 1371.2829113867579), ("['mul', 'add', 'mul', True, 8]", 1458.0786061675767), ("['mul', None, 'mul', False, 8]", 1483.4816529061632), ("['add', 'add', 'mul', False, 7]", 1594.3154807965213), ("['mul', 'add', 'mul', False, 7]", 1608.59971280795), ("['add', 'add', 'add', True, 8]", 1772.7873301618645), ("['mul', 'add', 'add', False, 4]", 1803.9858592614503), ("['add', 'add', 'mul', True, 8]", 1861.1571518605895), ("['mul', None, 'add', False, 9]", 1921.4454400532384), ("['add', 'add', 'add', False, 6]", 1953.038

#######################################################################
Neural Net with MAD is: 0.6676443769548925
LSTM with MAD is: 0.6012041642282996
#######################################################################
Neural Net with MAD is: 0.7526211275671515
CNN with MAD is: 0.7150795414064917
LSTM with MAD is: 0.9876257895444425
#######################################################################
Neural Net with MAD is: 1.03955524255977
CNN with MAD is: 0.6255240513895545
LSTM with MAD is: 1.000324928662637
#######################################################################
Neural Net with MAD is: 0.8509024365757497


KeyboardInterrupt: 

In [13]:

#import dill                            #pip install dill --user

import pickle
with open('dbase', 'wb') as f:
    pickle.dump(dbase, f)
with open('metadata', 'wb') as f:
    pickle.dump(metadat, f)    
with open('deleted_data', 'wb') as f:
    pickle.dump(Deleted_dat, f)     

    

In [9]:
import numpy as np
Data=[]
Scenarios=[]
Product=[]
Subproduct=[]
Actuals=[]
test_step=24
methods=40
mega=[]
a=[]
for i in range(len(metadat)):
    scen=metadat[i][0]
    product=metadat[i][1]
    subproduct=metadat[i][2]
    df=tableResult[(tableResult['Scenario']==scen) & (tableResult['Product']==product) &\
        (tableResult['Sub-Product']==subproduct)]
    df=df.sort_values(by='Date_time')
    df=df.reset_index()
    mdate=metadat[0][3][0]
    moddate=mdate.split("-")#[::-1]
    starts=moddate[0]
    moddate=moddate[0]+'-'+moddate[1]+'-'+moddate[2]
    dates = pd.to_datetime(pd.Series([moddate]), format = '%Y-%m-%d')  
    ends= pd.date_range(start=dates[0],periods=test_step,freq='MS')
    k=0
    for j in range(len(df)):
        if df['Date_time'][j]<dates[0]:
            meg=[df['Date_time'][j],scen,product,subproduct,df['Volumes'][j]]+[0 for _ in range(methods)]
            mega.append(meg)
        else:
            meg=[df['Date_time'][j],scen,product,subproduct,df['Volumes'][j]]+dbase[i][:,k].tolist()
            mega.append(meg)
            #print(meg)
            k+=1
    for m in range(k,test_step):
        meg=[ends[m],scen,product,subproduct,0]+dbase[i][:,m].tolist()
        mega.append(meg)
    a.append(len(mega))
more_col=['AutoRegressive_m','MovingAverage_m', 'AutoRegressive_MovingAverage_m' ,\
                                         'AutoRegressive_Integrated_MovingAverage_m',\
                                         'Seasonal_AutoRegressive_Integrated_MovingAverage_m1' ,\
                                         'Seasonal_AutoRegressive_Integrated_MovingAverage_m2' ,\
                                        'Seasonal_AutoRegressive_Integrated_MovingAverage_m3' ,\
                                        'Seasonal_AutoRegressive_Integrated_MovingAverage_m4' ,\
                                        'Seasonal_AutoRegressive_Integrated_MovingAverage_m5' ,\
                                         'Holts_added_linear_trend_m' ,\
                                         'Holts_added_damped_linear_trend_m' ,\
                                         'Holts_multiplicative_linear_trend_m'  ,\
                                         'Holts_multiplicative_damped_linear_trend_m' ,\
                                         'Holts_added_linear_trend_with_seasonality_m' ,\
                                         'Holts_added_dampened_linear_trend_with_seasonality_m' ,\
                                         'Holts_multiplicative_linear_trend_with_seasonality_m' ,\
                                         'Holts_multiplicative_dampened_linear_trend_with_seasonality_m' ,\
                                         'Holts_tunned_model_m1' ,\
                                        'Holts_tunned_model_m2' ,\
                                        'Holts_tunned_model_m3' ,\
                                         'Simple_Exponential_Smoothing_m' ,\
                                        'Error_trend_seasonality_tunned_m1' ,\
                                         'Error_trend_seasonality_tunned_m2' ,\
                                        'Error_trend_seasonality_tunned_m3' ,'NeuralNetwork_1m' , 'LongShort_termMemory_1m' , 'NeuralNetwork_2m',\
                                         'ConvolutionalNeuralNetwork_2m' ,'LongShort_termMemory_2m' ,'NeuralNetwork_3m' ,'ConvolutionalNeuralNetwork_3m' ,'LongShort_termMemory_3m' ,\
                                         'NeuralNetwork_4m' ,'ConvolutionalNeuralNetwork_4m' ,\
                                         'LongShort_termMemory_4m' ,'ConvolutionalNeuralNetwork_LongShort_termMemory_4m' , 'Prophet' ,'GPy',\
                                        'Decision_tree' ,'Random_forest']     
df2 = pd.DataFrame(mega,columns =['Date','Scenario','Product','Sub-Product', 'Actual Volumes']+more_col)  


In [10]:
df2

,Date,Scenario,Product,Sub-Product,Actual Volumes,AutoRegressive_m,MovingAverage_m,AutoRegressive_MovingAverage_m,AutoRegressive_Integrated_MovingAverage_m,Seasonal_AutoRegressive_Integrated_MovingAverage_m1,...,ConvolutionalNeuralNetwork_3m,LongShort_termMemory_3m,NeuralNetwork_4m,ConvolutionalNeuralNetwork_4m,LongShort_termMemory_4m,ConvolutionalNeuralNetwork_LongShort_termMemory_4m,Prophet,GPy,Decision_tree,Random_forest
0,2019-01-01,Field Ops - Mass Markets - Test 7/7/2021,SOUTHWEST,AZ,45565,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2019-02-01,Field Ops - Mass Markets - Test 7/7/2021,SOUTHWEST,AZ,39094,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2019-03-01,Field Ops - Mass Markets - Test 7/7/2021,SOUTHWEST,AZ,41412,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,2019-04-01,Field Ops - Mass Markets - Test 7/7/2021,SOUTHWEST,AZ,39175,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,2019-05-01,Field Ops - Mass Markets - Test 7/7/2021,SOUTHWEST,AZ,39036,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2041,2022-08-01,Service Delivery - RBO + SDP + EASE - Te,Wavelengths,Install,0,461.340732,415.657498,426.979285,426.979285,374.618194,...,458.619568,477.377563,391.738739,484.590576,486.107574,508.821594,680.992195,449.454545,449.454545,451.159762
2042,2022-09-01,Service Delivery - RBO + SDP + EASE - Te,Wavelengths,Install,0,462.676703,415.657498,444.462409,444.462409,-617.474338,...,458.653595,474.575073,534.528381,501.259399,486.323547,515.902710,688.735068,449.454545,449.454545,434.765262
2043,2022-10-01,Service Delivery - RBO + SDP + EASE - Te,Wavelengths,Install,0,463.168451,415.657498,445.004300,445.004300,538.592173,...,458.643311,474.633301,392.398651,495.077637,486.324310,500.517303,696.228171,449.454545,449.454545,546.418819
2044,2022-11-01,Service Delivery - RBO + SDP + EASE - Te,Wavelengths,Install,0,461.935487,415.657498,447.602670,447.602670,198.653445,...,458.656158,486.434540,533.656250,501.942444,486.352325,505.382782,703.971044,449.454545,449.454545,435.022577


In [11]:
write_df_to_sql(expID= "", new_df= df2, table_name='zzz_mithun_juptest', cnx= conn, engine=engine,
        replace_exist_table=False)